In [1]:
#Importing Libraries
# please do go through this python notebook: 
import warnings
warnings.filterwarnings("ignore")

import csv
import pandas as pd#pandas to create small dataframes 
import datetime #Convert to unix time
import time #Convert to unix time
# if numpy is not installed already : pip3 install numpy
import numpy as np#Do aritmetic operations on arrays
# matplotlib: used to plot graphs
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns#Plots
from matplotlib import rcParams#Size of plots  
from sklearn.cluster import MiniBatchKMeans, KMeans#Clustering
import math
import pickle
import os
# to install xgboost: pip3 install xgboost
import xgboost as xgb

import warnings
import networkx as nx
import pdb
import pickle
from pandas import HDFStore,DataFrame
from pandas import read_hdf
from scipy.sparse.linalg import svds, eigs
import gc
from tqdm import tqdm



In [2]:


if os.path.isfile('/home/rohit/projects/fb_link_prediction/after_eda/train_pos_after_eda.csv'):
    train_graph=nx.read_edgelist('/home/rohit/projects/fb_link_prediction/after_eda/train_pos_after_eda.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int)
    print(nx.info(train_graph))
else:
    print("please run the FB_EDA.ipynb or download the files from drive")



DiGraph with 1780722 nodes and 7550015 edges


In [3]:
#for followees
def jaccard_for_followees(a,b):
    try:
        if len(set(train_graph.successors(a))) == 0  | len(set(train_graph.successors(b))) == 0:
            return 0
        sim = (len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b)))))/\
                                    (len(set(train_graph.successors(a)).union(set(train_graph.successors(b)))))
    except:
        return 0
    return sim

In [4]:
#one test case
print(jaccard_for_followees(273084,1505602))

0.0


In [5]:


#node 1635354 not in graph 
print(jaccard_for_followees(273084,1505602))



0.0


In [6]:


#for followers
def jaccard_for_followers(a,b):
    try:
        if len(set(train_graph.predecessors(a))) == 0  | len(set(g.predecessors(b))) == 0:
            return 0
        sim = (len(set(train_graph.predecessors(a)).intersection(set(train_graph.predecessors(b)))))/\
                                 (len(set(train_graph.predecessors(a)).union(set(train_graph.predecessors(b)))))
        return sim
    except:
        return 0



In [7]:
print(jaccard_for_followers(273084,470294))

0


In [8]:
#node 1635354 not in graph 
print(jaccard_for_followees(669354,1635354))

0


In [9]:


#for followees
def cosine_for_followees(a,b):
    try:
        if len(set(train_graph.successors(a))) == 0  | len(set(train_graph.successors(b))) == 0:
            return 0
        sim = (len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b)))))/\
                                    (math.sqrt(len(set(train_graph.successors(a)))*len((set(train_graph.successors(b))))))
        return sim
    except:
        return 0



In [10]:


def cosine_for_followers(a,b):
    try:
        
        if len(set(train_graph.predecessors(a))) == 0  | len(set(train_graph.predecessors(b))) == 0:
            return 0
        sim = (len(set(train_graph.predecessors(a)).intersection(set(train_graph.predecessors(b)))))/\
                                     (math.sqrt(len(set(train_graph.predecessors(a))))*(len(set(train_graph.predecessors(b)))))
        return sim
    except:
        return 0



In [11]:
print(cosine_for_followers(2,470294))

0.02886751345948129


In [12]:
print(cosine_for_followers(669354,1635354))

0


In [13]:


if not os.path.isfile('/home/rohit/projects/fb_link_prediction/data/fea_sample/page_rank.p'):
    pr = nx.pagerank(train_graph, alpha=0.85)
    pickle.dump(pr,open('/home/rohit/projects/fb_link_prediction/data/fea_sample/page_rank.p','wb'))
else:
    pr = pickle.load(open('/home/rohit/projects/fb_link_prediction/data/fea_sample/page_rank.p','rb'))



In [14]:
print('min',pr[min(pr, key=pr.get)])
print('max',pr[max(pr, key=pr.get)])
print('mean',float(sum(pr.values())) / len(pr))

min 1.6556497245737814e-07
max 2.709825134193587e-05
mean 5.615699699389075e-07


In [15]:


#for imputing to nodes which are not there in Train data
mean_pr = float(sum(pr.values())) / len(pr)
print(mean_pr)



5.615699699389075e-07


In [16]:


#if has direct edge then deleting that edge and calculating shortest path
def compute_shortest_path_length(a,b):
    p=-1
    try:
        if train_graph.has_edge(a,b):
            train_graph.remove_edge(a,b)
            p= nx.shortest_path_length(train_graph,source=a,target=b)
            train_graph.add_edge(a,b)
        else:
            p= nx.shortest_path_length(train_graph,source=a,target=b)
        return p
    except:
        return -1



In [17]:
compute_shortest_path_length(77697, 826021)

10

In [18]:
compute_shortest_path_length(669354,1635354)

-1

In [19]:
#getting weekly connected edges from graph 
wcc=list(nx.weakly_connected_components(train_graph))
def belongs_to_same_wcc(a,b):
    index = []
    if train_graph.has_edge(b,a):
        return 1
    if train_graph.has_edge(a,b):
            for i in wcc:
                if a in i:
                    index= i
                    break
            if (b in index):
                train_graph.remove_edge(a,b)
                if compute_shortest_path_length(a,b)==-1:
                    train_graph.add_edge(a,b)
                    return 0
                else:
                    train_graph.add_edge(a,b)
                    return 1
            else:
                return 0
    else:
            for i in wcc:
                if a in i:
                    index= i
                    break
            if(b in index):
                return 1
            else:
                return 0

In [20]:
belongs_to_same_wcc(861, 1659750)

0

In [21]:
belongs_to_same_wcc(669354,1635354)

0

In [22]:
#adar index
def calc_adar_in(a,b):
    sum=0
    try:
        n=list(set(train_graph.successors(a)).intersection(set(train_graph.successors(b))))
        if len(n)!=0:
            for i in n:
                sum=sum+(1/np.log10(len(list(train_graph.predecessors(i)))))
            return sum
        else:
            return 0
    except:
        return 0


In [23]:
calc_adar_in(1,189226)

0

In [24]:
calc_adar_in(669354,1635354)

0

In [25]:
def follows_back(a,b):
    if train_graph.has_edge(b,a):
        return 1
    else:
        return 0


In [26]:
follows_back(1,189226)

1

In [27]:
follows_back(669354,1635354)

0

In [28]:
if not os.path.isfile('/home/rohit/projects/fb_link_prediction/data/fea_sample/katz.p'):
    katz = nx.katz.katz_centrality(train_graph,alpha=0.005,beta=1)
    pickle.dump(katz,open('/home/rohit/projects/fb_link_prediction/data/fea_sample/katz.p','wb'))
else:
    katz = pickle.load(open('/home/rohit/projects/fb_link_prediction/data/fea_sample/katz.p','rb'))

In [29]:
from sklearn.cluster import KMeans
import numpy as np
X_cluster = np.array([[1, 2], [1, 4], [1, 0],
              [10, 2], [10, 4], [10, 0]])
kmeans = KMeans(n_clusters=2, random_state=0).fit(X_cluster)
kmeans.labels_

kmeans.predict([[0, 0], [12, 3]])

kmeans.cluster_centers_

array([[10.,  2.],
       [ 1.,  2.]])

In [30]:
kmeans.labels_

array([1, 1, 1, 0, 0, 0], dtype=int32)

In [31]:
kmeans.predict([[0, 0], [12, 3]])

array([1, 0], dtype=int32)